# Téléchargement depuis le FTP INPI 

L\'objectif de ce notebook est de récupérer la donnée de l\'INPI provenant du FTP

Le téléchargement depuis le FTP INPI a plusieurs contraintes :

- la connexion FTP est bloquée par le proxy sur un poste standard CALF.
- il y a un quota de téléchargement de 3Go par jour.
- la connexion est coupée automatiquement par l'hôte au bout d'une centaine de fichiers récupérés via un script.
- l'arborescence dans le FTP est profonde et pas incrémentale car liée à la codification des greffes. Ex :
/public
    /IMR_Donnees_Saisies
        /tc
            /flux
                /2017
                    /08
                        /04
                            /0101
                                /57
                            /0202
                                /60
                            /0203
                                /58
                            /0301
                                /58
                            ..etc
                            
Ce notebook est basé sur la librairie ftputil et un parcours sur mesure de l'arborescence FTP INPI. Il ne fonctionne donc pas sur un autre FTP ou avec des sous-dossiers de ce FTP.

Fonctions :
- Se connecter au FTP
- Télécharger un fichier depuis le FTP
- Télécharger tous les fichiers d'un dossier spécifique depuis le FTP

# Connect to FTP

In [ ]:
# INPI FTP Connexion parameters
host = 'opendata-rncs.inpi.fr'
port = 21
usr = 'calf'
pwd = 'Calfinpi2019'

In [ ]:
# Connect to FTP
def connect_inpi(host,port,usr,pwd):
    ftps = ftplib.FTP_TLS()
    ftps.connect(host, port)
    ftps.login(usr, pwd)
    ftps.prot_p()
    return ftps

In [ ]:
ftps = connect_inpi(host,port,usr,pwd)

# Download single file

In [ ]:
# Download file
def download_file(ftps,path,name):
    try:
        fullname = "{0}/{1}".format(path,name)
        with open(name, 'wb' ) as file :
            print(fullname)
            ftps.retrbinary('RETR %s' % fullname, file.write)
            return True
    except Exception as e:
        print('Error: ' + str(e))
        return False

In [ ]:
path =  '/public/IMR_Donnees_Saisies/tc/flux/2017/05/18/0101/1'
name = '0101_1_20170512_112544_10_ets_supprime_EVT.csv'
download_file(ftps,path,name)

# Download all files from path

In [ ]:
path_FTP='/public/IMR_Donnees_Saisies/tc/stock/2017'

In [ ]:
def download_path(ftps,path_FTP):
    
    ftps.cwd(path_FTP) 
    list_dir_ftp = ftps.nlst()
    regex = r"([^\/]+$)"

    #list all dir in the FTP directory
    list_dir_ftp = ftps.nlst()
        
    # Loop into sub-folders
    for f in list_dir_ftp:
        date = ftps.nlst(f)
        for sf in date:
            file = ftps.nlst(sf)
            for greffe in file:
                next = ftps.nlst(greffe)
                for sff in next:
                    fcsv = ftps.nlst(sff)
                    for rel_filename in fcsv:
                        if rel_filename.endswith(('.csv','.zip')):
                            full_name = "{0}/{1}".format(path_FTP,rel_filename)
                            filename = rel_filename.split("/")[-1]
                            filepath=full_name[:len(full_name)-len(filename)]
                            download_file(ftps,filepath,filename)
    ftps.quit()

In [ ]:
download_path(ftps,path_FTP)